In [2]:
# Import Dependencies
import requests
from config import api_key
import censusdata
import pandas as pd  
from sqlalchemy import create_engine
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
import numpy as np
from sqlalchemy import func
from sqlalchemy import distinct
import json
import psycopg2
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
cloud_engine = create_engine(f"postgresql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
cloud_conn = cloud_engine.connect()

ca_school_data = pd.read_sql('select * from sd_master_df', cloud_conn)

ca_school_data.head()

,zipcode,Total Students Enrolled,Average_School_Rating,MEDIAN HOUSEHOLD INCOME,MEDIAN INCOME BLACK/AA,MEDIAN INCOME INDIGENOUS PEOPLE,MEDIAN INCOME ASIAN,MEDIAN INCOME HAW/PAC ISLANDER,MEDIAN INCOME OTHER RACE,MEDIAN INCOME MIXED RACE,...,population_haw_pac_islander_ratio,population_other_ratio,population_mixed_ratio,population_hispanic_latino_ratio,Theft Count,latitude,longitude,Substance Abuse Count,Violent Count,Miscellaneous Count
0,91901,1560.0,5.953846,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,...,0.001297,0.006754,0.012212,0.168153,79.0,32.78992,-116.71120,317.0,50.0,52.0
1,91902,1372.0,6.148688,86913.0,113875.0,0.0,80192.0,0.0,68750.0,197708.0,...,0.004943,0.000000,0.036039,0.459204,65.0,32.67400,-117.02100,24.0,19.0,29.0
2,91906,454.0,4.279736,49276.0,0.0,24306.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.015291,0.260242,11.0,32.70247,-116.50462,4.0,15.0,6.0
3,91910,15136.0,6.237381,53890.0,54475.0,43287.0,69688.0,85417.0,56894.0,55625.0,...,0.003635,0.001798,0.025032,0.599635,645.0,32.63700,-117.05000,251.0,304.0,273.0
4,91911,14330.0,5.293859,49156.0,42902.0,36932.0,77083.0,0.0,50487.0,50938.0,...,0.005052,0.000499,0.011816,0.740116,561.0,32.61600,-117.03400,209.0,205.0,181.0


Four New Columns for Crime ratios are created which will standardize the count with the Zipcode population
The new columns created are 'Theft Count Ratio', 'Substance Abuse Count Ratio', 'Violent Count Ratio' and 'Miscellaneous Count Ratio'

In [4]:
ca_school_data['Theft Count Ratio'] = ca_school_data['Theft Count'] / ca_school_data['POPULATION_TOTAL']
ca_school_data['Substance Abuse Count Ratio'] = ca_school_data['Substance Abuse Count'] / ca_school_data['POPULATION_TOTAL']
ca_school_data['Violent Count Ratio'] = ca_school_data['Violent Count'] / ca_school_data['POPULATION_TOTAL']
ca_school_data['Miscellaneous Count Ratio'] = ca_school_data['Miscellaneous Count'] / ca_school_data['POPULATION_TOTAL']
ca_school_data.head(75)

,zipcode,Total Students Enrolled,Average_School_Rating,MEDIAN HOUSEHOLD INCOME,MEDIAN INCOME BLACK/AA,MEDIAN INCOME INDIGENOUS PEOPLE,MEDIAN INCOME ASIAN,MEDIAN INCOME HAW/PAC ISLANDER,MEDIAN INCOME OTHER RACE,MEDIAN INCOME MIXED RACE,...,Theft Count,latitude,longitude,Substance Abuse Count,Violent Count,Miscellaneous Count,Theft Count Ratio,Substance Abuse Count Ratio,Violent Count Ratio,Miscellaneous Count Ratio
0,91901,1560.0,5.953846,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,...,79.0,32.78992,-116.71120,317.0,50.0,52.0,0.004269,0.017129,0.002702,0.002810
1,91902,1372.0,6.148688,86913.0,113875.0,0.0,80192.0,0.0,68750.0,197708.0,...,65.0,32.67400,-117.02100,24.0,19.0,29.0,0.003455,0.001276,0.001010,0.001541
2,91906,454.0,4.279736,49276.0,0.0,24306.0,0.0,0.0,0.0,0.0,...,11.0,32.70247,-116.50462,4.0,15.0,6.0,0.003174,0.001154,0.004328,0.001731
3,91910,15136.0,6.237381,53890.0,54475.0,43287.0,69688.0,85417.0,56894.0,55625.0,...,645.0,32.63700,-117.05000,251.0,304.0,273.0,0.008344,0.003247,0.003933,0.003532
4,91911,14330.0,5.293859,49156.0,42902.0,36932.0,77083.0,0.0,50487.0,50938.0,...,561.0,32.61600,-117.03400,209.0,205.0,181.0,0.006669,0.002484,0.002437,0.002152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,92116,1583.0,6.320278,56731.0,51123.0,0.0,55163.0,0.0,35425.0,45950.0,...,228.0,32.76500,-117.12800,29.0,50.0,84.0,0.006886,0.000876,0.001510,0.002537
71,92117,6194.0,5.128350,65325.0,39485.0,78090.0,73320.0,49124.0,51560.0,68125.0,...,347.0,32.81800,-117.19700,80.0,99.0,117.0,0.006489,0.001496,0.001851,0.002188
72,92118,3044.0,9.106767,90256.0,0.0,0.0,112500.0,0.0,33750.0,72212.0,...,124.0,32.67172,-117.16443,36.0,26.0,66.0,0.005406,0.001569,0.001133,0.002877
73,92119,2280.0,7.090351,71071.0,36875.0,0.0,77875.0,0.0,39306.0,0.0,...,117.0,32.81600,-117.03300,13.0,23.0,38.0,0.004822,0.000536,0.000948,0.001566


Removing the Crime Actual Counts from the dataframe i.e. 
1. Theft Count
2. Substance Abuse Count
3. Violent Count
4. Miscellaneous Count

In [5]:
ca_school_data = ca_school_data.drop(labels=["Theft Count", "Substance Abuse Count","Violent Count","Miscellaneous Count"], axis=1)
ca_school_data.head(75)

,zipcode,Total Students Enrolled,Average_School_Rating,MEDIAN HOUSEHOLD INCOME,MEDIAN INCOME BLACK/AA,MEDIAN INCOME INDIGENOUS PEOPLE,MEDIAN INCOME ASIAN,MEDIAN INCOME HAW/PAC ISLANDER,MEDIAN INCOME OTHER RACE,MEDIAN INCOME MIXED RACE,...,population_haw_pac_islander_ratio,population_other_ratio,population_mixed_ratio,population_hispanic_latino_ratio,latitude,longitude,Theft Count Ratio,Substance Abuse Count Ratio,Violent Count Ratio,Miscellaneous Count Ratio
0,91901,1560.0,5.953846,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,...,0.001297,0.006754,0.012212,0.168153,32.78992,-116.71120,0.004269,0.017129,0.002702,0.002810
1,91902,1372.0,6.148688,86913.0,113875.0,0.0,80192.0,0.0,68750.0,197708.0,...,0.004943,0.000000,0.036039,0.459204,32.67400,-117.02100,0.003455,0.001276,0.001010,0.001541
2,91906,454.0,4.279736,49276.0,0.0,24306.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.015291,0.260242,32.70247,-116.50462,0.003174,0.001154,0.004328,0.001731
3,91910,15136.0,6.237381,53890.0,54475.0,43287.0,69688.0,85417.0,56894.0,55625.0,...,0.003635,0.001798,0.025032,0.599635,32.63700,-117.05000,0.008344,0.003247,0.003933,0.003532
4,91911,14330.0,5.293859,49156.0,42902.0,36932.0,77083.0,0.0,50487.0,50938.0,...,0.005052,0.000499,0.011816,0.740116,32.61600,-117.03400,0.006669,0.002484,0.002437,0.002152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,92116,1583.0,6.320278,56731.0,51123.0,0.0,55163.0,0.0,35425.0,45950.0,...,0.000000,0.002416,0.037964,0.261250,32.76500,-117.12800,0.006886,0.000876,0.001510,0.002537
71,92117,6194.0,5.128350,65325.0,39485.0,78090.0,73320.0,49124.0,51560.0,68125.0,...,0.006264,0.002431,0.040859,0.219945,32.81800,-117.19700,0.006489,0.001496,0.001851,0.002188
72,92118,3044.0,9.106767,90256.0,0.0,0.0,112500.0,0.0,33750.0,72212.0,...,0.001875,0.001264,0.029339,0.143511,32.67172,-117.16443,0.005406,0.001569,0.001133,0.002877
73,92119,2280.0,7.090351,71071.0,36875.0,0.0,77875.0,0.0,39306.0,0.0,...,0.001731,0.001690,0.043396,0.170534,32.81600,-117.03300,0.004822,0.000536,0.000948,0.001566


Removing the following columns from the dataframe i.e. 
1. latitude
2. longitude

In [6]:
ca_school_drop = ca_school_data.drop(labels=["latitude", "longitude"], axis=1)
ca_school_drop.head(75)

,zipcode,Total Students Enrolled,Average_School_Rating,MEDIAN HOUSEHOLD INCOME,MEDIAN INCOME BLACK/AA,MEDIAN INCOME INDIGENOUS PEOPLE,MEDIAN INCOME ASIAN,MEDIAN INCOME HAW/PAC ISLANDER,MEDIAN INCOME OTHER RACE,MEDIAN INCOME MIXED RACE,...,population_indigenous_ratio,population_asian_ratio,population_haw_pac_islander_ratio,population_other_ratio,population_mixed_ratio,population_hispanic_latino_ratio,Theft Count Ratio,Substance Abuse Count Ratio,Violent Count Ratio,Miscellaneous Count Ratio
0,91901,1560.0,5.953846,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,...,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,0.004269,0.017129,0.002702,0.002810
1,91902,1372.0,6.148688,86913.0,113875.0,0.0,80192.0,0.0,68750.0,197708.0,...,0.000957,0.088449,0.004943,0.000000,0.036039,0.459204,0.003455,0.001276,0.001010,0.001541
2,91906,454.0,4.279736,49276.0,0.0,24306.0,0.0,0.0,0.0,0.0,...,0.062320,0.011829,0.000000,0.000000,0.015291,0.260242,0.003174,0.001154,0.004328,0.001731
3,91910,15136.0,6.237381,53890.0,54475.0,43287.0,69688.0,85417.0,56894.0,55625.0,...,0.001514,0.098251,0.003635,0.001798,0.025032,0.599635,0.008344,0.003247,0.003933,0.003532
4,91911,14330.0,5.293859,49156.0,42902.0,36932.0,77083.0,0.0,50487.0,50938.0,...,0.001426,0.065619,0.005052,0.000499,0.011816,0.740116,0.006669,0.002484,0.002437,0.002152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,92116,1583.0,6.320278,56731.0,51123.0,0.0,55163.0,0.0,35425.0,45950.0,...,0.004168,0.048505,0.000000,0.002416,0.037964,0.261250,0.006886,0.000876,0.001510,0.002537
71,92117,6194.0,5.128350,65325.0,39485.0,78090.0,73320.0,49124.0,51560.0,68125.0,...,0.004376,0.107934,0.006264,0.002431,0.040859,0.219945,0.006489,0.001496,0.001851,0.002188
72,92118,3044.0,9.106767,90256.0,0.0,0.0,112500.0,0.0,33750.0,72212.0,...,0.002616,0.026505,0.001875,0.001264,0.029339,0.143511,0.005406,0.001569,0.001133,0.002877
73,92119,2280.0,7.090351,71071.0,36875.0,0.0,77875.0,0.0,39306.0,0.0,...,0.000618,0.053287,0.001731,0.001690,0.043396,0.170534,0.004822,0.000536,0.000948,0.001566


In [7]:
# Using OneHotEncoder to create the features
# Generate our categorical variable lists
ca_school_cat = list(ca_school_drop.dtypes[ca_school_drop.dtypes == "object"].index)
ca_school_cat

['zipcode']

In [17]:
ca_school_drop["zipcode"] = ca_school_drop["zipcode"].astype(str).astype(int)

ca_school_notnull_df = ca_school_drop.dropna()
ca_school_notnull_df

,zipcode,Total Students Enrolled,Average_School_Rating,MEDIAN HOUSEHOLD INCOME,MEDIAN INCOME BLACK/AA,MEDIAN INCOME INDIGENOUS PEOPLE,MEDIAN INCOME ASIAN,MEDIAN INCOME HAW/PAC ISLANDER,MEDIAN INCOME OTHER RACE,MEDIAN INCOME MIXED RACE,...,population_indigenous_ratio,population_asian_ratio,population_haw_pac_islander_ratio,population_other_ratio,population_mixed_ratio,population_hispanic_latino_ratio,Theft Count Ratio,Substance Abuse Count Ratio,Violent Count Ratio,Miscellaneous Count Ratio
0,91901,1560.0,5.953846,79859.0,143917.0,53750.0,78625.0,0.0,48026.0,0.0,...,0.018263,0.011023,0.001297,0.006754,0.012212,0.168153,0.004269,0.017129,0.002702,0.002810
1,91902,1372.0,6.148688,86913.0,113875.0,0.0,80192.0,0.0,68750.0,197708.0,...,0.000957,0.088449,0.004943,0.000000,0.036039,0.459204,0.003455,0.001276,0.001010,0.001541
2,91906,454.0,4.279736,49276.0,0.0,24306.0,0.0,0.0,0.0,0.0,...,0.062320,0.011829,0.000000,0.000000,0.015291,0.260242,0.003174,0.001154,0.004328,0.001731
3,91910,15136.0,6.237381,53890.0,54475.0,43287.0,69688.0,85417.0,56894.0,55625.0,...,0.001514,0.098251,0.003635,0.001798,0.025032,0.599635,0.008344,0.003247,0.003933,0.003532
4,91911,14330.0,5.293859,49156.0,42902.0,36932.0,77083.0,0.0,50487.0,50938.0,...,0.001426,0.065619,0.005052,0.000499,0.011816,0.740116,0.006669,0.002484,0.002437,0.002152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,92139,4554.0,6.125604,58050.0,56821.0,108333.0,65814.0,0.0,75204.0,57601.0,...,0.000731,0.286809,0.014783,0.000244,0.036552,0.411220,0.003439,0.000650,0.000948,0.001625
86,92154,14715.0,4.830921,60380.0,52000.0,75441.0,82632.0,84205.0,59250.0,68359.0,...,0.002163,0.126319,0.003024,0.000476,0.025744,0.673187,0.007203,0.001857,0.004462,0.002061
87,92173,4627.0,3.540739,35605.0,71250.0,0.0,68194.0,0.0,46225.0,38750.0,...,0.000000,0.014526,0.000185,0.000000,0.004234,0.947180,0.014403,0.003462,0.003276,0.003554
88,92592,16902.0,8.035262,89445.0,117550.0,105932.0,98504.0,0.0,78125.0,108542.0,...,0.013188,0.098773,0.003849,0.003307,0.043516,0.219026,0.000000,0.000013,0.000000,0.000013


In [18]:
ca_school_cat = list(ca_school_notnull_df.dtypes[ca_school_notnull_df.dtypes == "object"].index)
ca_school_cat

[]

In [24]:
# Split our preprocessed data into our features and target arrays
X = ca_school_notnull_df.drop(["Average_School_Rating","zipcode"], axis=1).values
y = ca_school_notnull_df["Average_School_Rating"].values.reshape(-1, 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
#hidden_nodes_layer3 = 300
#hidden_nodes_layer4 = 200
#hidden_nodes_layer5 = 100

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 80)                3200      
_________________________________________________________________
dense_10 (Dense)             (None, 30)                2430      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 31        
Total params: 5,661
Trainable params: 5,661
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adadelta", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
3/3 [==============================] - 0s 2ms/step - loss: -0.2861 - accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 1ms/step - loss: -0.2878 - accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 1ms/step - loss: -0.2898 - accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 3ms/step - loss: -0.2919 - accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 1ms/step - loss: -0.2935 - accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 996us/step - loss: -0.2955 - accuracy: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 0s 1ms/step - loss: -0.2977 - accuracy: 0.0000e+00
Epoch 8/50
3/3 [==============================] - 0s 1ms/step - loss: -0.2994 - accuracy: 0.0000e+00
Epoch 9/50
3/3 [==============================] - 0s 1ms/step - loss: -0.3011 - accuracy: 0.0000e+00
Epoch 10/50
3/3 [==============================] - 0s 1ms/step - loss: -0.3026 - accuracy

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: -4.0272e-01 - accuracy: 0.0000e+00
Loss: -0.40271511673927307, Accuracy: 0.0
